## Logistic Version of the Isolet Model Test in Wallaroo

The following demonstrates how to use a converted sk-learn Logistics ML model converted to ONNX  in Wallaroo.  This is meant to be a follow up to the tutorial `convert-sklearn-logistic-to-onnx.ipyb`, which demonstrates how to convert the sk-learn logicistics model to ONNX format and generates the file `isolet_logistic_model_numclass.onnx` that is used in this tutorial.

For more details on using Wallaroo workspaces, pipelines, and models, see the [Wallaroo Documentation site](https://docs.wallaroo.ai).

This tutorial includes the following assets:

* `isolet_test_data.tsv`:  A test file that can be used to verify the output of the converted logistic model.
* `test-converted-sklearn-logistics-to-onnx.ipynb`: This Jupyter Notebook that demonstrates the use of the converted sk-learn logistic ML model in ONNX with Wallaroo.

## Test Steps

### Load Libraries

The first step in the process is to load the necessary libraries.

In [13]:
# from their artifacts

import json
import wallaroo
import numpy
import pandas

### Connect to Wallaroo Instance

Next we'll connect to the Wallaroo Instance with the `Client` method.

In [15]:
wl = wallaroo.Client()

### Connect to Workspace

Either create or use an existing workspace.  In this example, we will connect to the existing workspace `isolettest` by using the `set_current_workspace` command.  For more details on connecting to a workspace, see the [Wallaroo SDK Essentials guide](https://docs.wallaroo.ai/wallaroo-sdk/wallaroo-sdk-essentials-guide/).

![Wallaroo Workspace Example](images/isolet-workspace-example.png)

In [16]:
wl.set_current_workspace(wl.list_workspaces()[1])

{'name': 'isolettest', 'id': 2, 'archived': False, 'created_by': 'e435a290-8912-41c3-9d60-a0c8652a0128', 'created_at': '2022-06-17T20:22:53.577601+00:00', 'models': [{'name': 'isolet-test', 'version': '43dafdd7-e254-43db-8bff-ab54c1b3579f', 'file_name': 'isolet_logistic_model_numclass.onnx', 'last_update_time': datetime.datetime(2022, 6, 17, 20, 25, 30, 494597, tzinfo=tzutc())}, {'name': 'isolet-test', 'version': '621f4e1b-5fb1-48d8-97a8-49f2422c45fb', 'file_name': 'isolet_logistic_model_numclass.onnx', 'last_update_time': datetime.datetime(2022, 6, 17, 20, 32, 5, 374160, tzinfo=tzutc())}, {'name': 'isolet-test', 'version': '211712fe-9a0c-4b5d-86cb-2d26438c8897', 'file_name': 'isolet_logistic_model_numclass.onnx', 'last_update_time': datetime.datetime(2022, 6, 17, 21, 41, 53, 701617, tzinfo=tzutc())}], 'pipelines': [{'name': 'isolet-pipeline', 'create_time': datetime.datetime(2022, 6, 17, 20, 26, 21, 335312, tzinfo=tzutc()), 'definition': '[]'}]}

### Upload Model

Upload the `isolet_logistic_model_numclass.onnx` model as `isolet-test` with the `upload_model` method:

In [17]:
wl.upload_model('isolet-test', './isolet_logistic_model_numclass.onnx').configure()

{'name': 'isolet-test', 'version': '8e6217ae-b994-4b96-8541-e98590bd9ed2', 'file_name': 'isolet_logistic_model_numclass.onnx', 'last_update_time': datetime.datetime(2022, 6, 21, 18, 50, 25, 445012, tzinfo=tzutc())}

### Deploy Pipeline

Pipelines can be deployed either with the Wallaroo Dashboard user interface or through the SDK.  For this example, we have the pipeline `isolet-pipeline` deployed and assigned to the variable `isoletpipeline`.  For more details, see the [Wallaroo Pipeline Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-pipeline-management/).

![Wallaroo Workspace Example](images/isolet-pipeline-example.png)

We can check the status of the pipeline with the Wallaroo pipeline `status` method.

In [18]:
isoletpipeline = wl.list_pipelines()[0]

In [19]:
isoletpipeline.status()

{'status': 'Running',
 'details': None,
 'engines': [{'ip': '10.164.1.167',
   'name': 'engine-565d945cbb-7lxxl',
   'status': 'Running',
   'reason': None,
   'pipeline_statuses': {'pipelines': [{'id': 'isolet-pipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'isolet-test',
      'version': '43dafdd7-e254-43db-8bff-ab54c1b3579f',
      'sha': 'ce84b862cacf749755a93cec97232372a56af9138c960102f3cd4a4cfb491489',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.164.5.30',
   'name': 'engine-lb-85846c64f8-cfkkm',
   'status': 'Running',
   'reason': None}]}

### Load the Data

Load the data from the file `isolet_test_data.tsv` into the variable `testdata`, and turn it into a data dictionary variable labeled `input_dict`.

In [20]:
# read in data
testdata = pandas.read_csv('isolet_test_data.tsv', sep='\t')

input_dict = {
    'tensor': testdata.values.tolist()
}

We'll display the data shape to verify it has 9 rows and 617 columsn per row.

In [21]:
# report the size of the data  -  9 rows, 617 columns each
testdata.shape

(9, 617)

### Run Inference

Use the Wallaroo pipeline `infer` method to run an inference and store the results into the variable `result`, then display the data from the result.

In [22]:
result = isoletpipeline.infer(input_dict)

In [23]:
result[0].data()

[array([0.e+000, 0.e+000, 5.e-324, 5.e-324, 0.e+000, 0.e+000, 5.e-324,
        5.e-324, 0.e+000]),
 array([[9.99962986e-01, 3.70144844e-05],
        [9.97749865e-01, 2.25013494e-03],
        [4.73588705e-03, 9.95264113e-01],
        [3.10786664e-02, 9.68921304e-01],
        [9.99630690e-01, 3.69310379e-04],
        [9.99202728e-01, 7.97271729e-04],
        [1.47812575e-01, 8.52187395e-01],
        [8.04565549e-02, 9.19543445e-01],
        [9.98787403e-01, 1.21256709e-03]])]

### Closing

Once the tests are done, the pipeline should be undeployed to allocate the resources back to the Wallaroo instance.  For more details, see the [Wallaroo Pipeline Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-pipeline-management/).